## Assignment 3: Bond Pricing
*By: Ashish Mathew*

In [3]:
#!pip install pandas numpy scipy seaborn matplotlib yfinance scikit-learn

In [4]:
from datetime import datetime
from scipy.optimize import newton

### Bond Pricing Formula

The price \( P \) of a bond is the present value of its future cash flows, which include periodic coupon payments and the face value at maturity.

$
P = \sum_{t=1}^{N} \frac{C}{(1 + r)^t} + \frac{F}{(1 + r)^N}
$

Where:

- \( P \) = Price of the bond  
- \( C \) = Coupon payment per period  
- \( r \) = Discount rate (market interest rate per period)  
- \( N \) = Number of periods until maturity  
- \( F \) = Face value (par value) of the bond 

#### Present Value of Coupon Payments (Using Geometric Progression)

If a bond pays a fixed coupon \( C \) per period for \( N \) periods, and the discount rate per period is \( r \), the present value of all coupon payments is given by:

$
PV_{\text{coupons}} = C \cdot \sum_{t=1}^{N} \frac{1}{(1 + r)^t}
$

This is a finite geometric series with first term $( a = \frac{1}{1 + r} )$ and common ratio $( q = \frac{1}{1 + r} )$, so the closed-form formula is:

$
PV_{\text{coupons}} = C \cdot \left( \frac{1 - (1 + r)^{-N}}{r} \right)
$

Where:

- \( C \) = Coupon payment per period  
- \( r \) = Discount rate per period  
- \( N \) = Number of periods  

In [5]:
def CalculateBondPV(CouponRate, FaceValue, YTM, YearsToMaturity, CouponsPerYear):
    """
    Calculate the present value (price) of a bond.

    Parameters:
    - CouponRate (float): Annual coupon rate (as a decimal, e.g., 0.05 for 5%)
    - FaceValue (float): Face or par value of the bond (e.g., 1000)
    - YTM (float): Annual yield to maturity (as a decimal, e.g., 0.04 for 4%)
    - YearsToMaturity (int or float): Time remaining until the bond matures (in years)
    - CouponsPerYear (int): Number of coupon payments per year (e.g., 2 for semi-annual)

    Returns:
    - float: Present value (price) of the bond
    """

    # Calculate the coupon payment per period.
    # CouponRate is annual, so divide by CouponsPerYear.
    coupon_payment = CouponRate * FaceValue / CouponsPerYear

    # Total number of periods (e.g., 2 years * 2 semiannual = 4 periods)
    periods = YearsToMaturity * CouponsPerYear

    # Periodic yield (YTM per period)
    r = YTM / CouponsPerYear

    # Present value of all coupon payments (geometric series formula)
    pv_coupons = coupon_payment * (1 - (1 + r) ** (-periods)) / r

    # Present value of the face value, discounted back from maturity
    pv_facevalue = FaceValue / (1 + r) ** periods

    # Total bond price = PV of coupons + PV of face value
    bond_value = pv_facevalue + pv_coupons

    return bond_value

In [6]:
def CalculateAccruedInterest(CouponRate, FaceValue, SettlementDate, LastCouponDate, CouponsPerYear):
    # Extract date components
    y1, m1, d1 = LastCouponDate.year, LastCouponDate.month, LastCouponDate.day
    y2, m2, d2 = SettlementDate.year, SettlementDate.month, SettlementDate.day

    # Apply 30/360 day count convention rules
    if d2 == 31 and d1 >= 30:
        d2 = 30
    if d1 == 31:
        d1 = 30
    
    # Calculate days between dates using 30/360 formula
    days = (360 * (y2 - y1) + 30 * (m2 - m1) + (d2 - d1))

    # Calculate days in coupon period (always 360/coupons_per_year in 30/360)
    days_in_period = 360 / CouponsPerYear
    # Calculate fraction of period
    fraction = days / days_in_period

    # Calculate accrued interest
    accrued_interest = FaceValue * CouponRate * fraction / CouponsPerYear

    return accrued_interest

In [7]:
def CalculateDirtyPrice(FaceValue, CouponRate, CouponsPerYear, YTM, YearsToMaturity, SettlementDate, LastCouponDate):
    clean_price = CalculateBondPV(CouponRate, FaceValue, YTM, YearsToMaturity, CouponsPerYear)

    accrued_interest = CalculateAccruedInterest(CouponRate, FaceValue, SettlementDate, LastCouponDate, CouponsPerYear)

    dirty_price = clean_price + accrued_interest

    return clean_price, accrued_interest, dirty_price

### GM Market Price Calculation

![alt-text](./assets/gm_bond.png)

In [8]:
FaceValue = 100
CouponRate = 5.65 / 100
YearsToMaturity = 3
YTM = 4.98 / 100
CouponsPerYear = 2
LastCouponDate = datetime(year = 2025, month=6, day=17)
SettlementDate = datetime(year=2026, month=1, day=17)

market_price = CalculateBondPV(CouponRate, FaceValue, YTM, YearsToMaturity, CouponsPerYear)

print(f"market price: ${market_price:.2f}")

market price: $101.85


In [9]:
# Function to solve for YTM
def YTM_solver(MarketPrice, CouponRate, FaceValue, YearsToMaturity, CouponsPerYear):
    """
    Solve for the Yield to Maturity (YTM) given market price and bond parameters.
    """

    def objective_function(YTM_guess):
        # Calculates the difference between calculated price and market price
        return CalculateBondPV(CouponRate, FaceValue, YTM_guess, YearsToMaturity, CouponsPerYear) - MarketPrice

    # Use Newton's method to find the root (YTM that gives the correct price)
    ytm = newton(objective_function, x0=0.05)  # Initial guess = 5%
    return ytm

### AMD Bond Yield-to-Maturity Calculation

![alt-text](./assets/amd_bond.png)

In [10]:
MarketPrice = 100.10
CouponRate = 4.212 / 100
FaceValue = 100
YearsToMaturity = 1
CouponsPerYear = 2

ytm = YTM_solver(MarketPrice, CouponRate, FaceValue, YearsToMaturity, CouponsPerYear)
print(f"Yield to maturity is {ytm*100:.2f}%")

Yield to maturity is 4.11%
